# Comparisons between MR fields

Steps:   
1. CLEAN DATA
1. SELECT SESSIONS
1. ANALYSES
    - (visualize unsmoothed, smoothed maps)
    - within study TLE vs CTRL comparison
        - extract smoother maps
        - compute z, w scores (values per participant)
        - group and flip
        - Cohen's D (compare TLE and control z/w score distributions within each vertex)
    - between study 7T vs 3T comparison      


In [ ]:
import os
import sys
sys.path.append(os.getcwd())
import pandas as pd
import importlib
import tTsTGrpUtils as tsutil
import utils_plots as uplots

lab = True
save = True
verbose = True
toPrint = True

test = False
test_frac = 0.1 # fraction of demo to use for testing if test=True

includeBL = False # if should include bilateral TLE patients (with one side higher than other) in analyses

if lab: # define root paths to source files
    src_dir = "/host/verges/tank/data/daniel/01_3T7T/z/data/sources" # path to directory with source pt sheets
    sys.path.append("/host/verges/tank/data/daniel/")
    if save:
        save_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs"
else:
    src_dir = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/PT/sources" # path to directory with source pt sheets
    sys.path.append("/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/code")
    if save:
        save_pth = "/Users/danielmendelson/Library/CloudStorage/OneDrive-McGillUniversity/Documents/PhD/Boris/projects/3T7T/data/outputs"

##### ANALYSIS SPECIFICATIONS #####

# Demographics details
demographics = {
    "df_pths_qc_pth" : "/host/verges/tank/data/daniel/01_3T7T/z/outputs/03b_mapPths_QC_26Nov2025-125407.csv", # NOTE: path to demographics file with merged QC cols produced by 02_demo.ipynb
    # column names:
    'nStudies': True, # whether multiple studies are included
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date",
    "age": "age",
    "sex": "sex",
    "grp" : "grp_detailed" # col name for participant grouping variable to use
}

# Study details
MICs = {
    "name": "MICs",
    "dir_root": "/data/mica3/BIDS_MICs",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/zbrains_clinical",
    "study": "3T",
    "ID_ctrl" : ["HC"], # patterns for control IDs in demographics file
    "ID_Pt" : ["PX"] # patterns for patient IDs in demographics file
    }

PNI = {
    "name": "PNI",
    "dir_root": "/data/mica3/BIDS_PNI",
    "dir_raw": "/rawdata",
    "dir_deriv": "/derivatives",
    "dir_mp": "/micapipe_v0.2.0",
    "dir_hu": "/hippunfold_v1.3.0/hippunfold",
    "dir_zb": "/zbrains_clinical",
    "study": "7T",
    "ID_col" : ["PNC", "Pilot"], # column for ID in demographics file
    }

studies = [MICs, PNI]

ctrl_grp = {'ctrl' : ['CTRL']}

# Analysis details
specs  = { # all spec values to be in lists to allow for iteration across these values
    # directories
    'prjDir_root' : "/host/verges/tank/data/daniel/01_3T7T/z", 
    'prjDir_outs' : "/outputs",
    'prjDir_out_stats': "/outputs/stats",
    'prjDir_out_figs': "/outputs/figures",
    'prjDir_maps': "/maps", # output directory for smoothed cortical maps
    'prjDir_dictLists': "/maps/dictLists",
    'prjDir_mapPths' : "/output/paths",
    'prjDir_maps_dfs': "/outputs/dfs/04a_maps_dfs",
    'prjDir_parc_dfs': "/outputs/dfs/04b_maps_parc",
    'prjDir_winComp_dfs': "/outputs/dfs/05a_winComp",
    'prjDir_grpFlip_dfs': "/outputs/dfs/05b_grpFlip",
    'prjDir_winD_dfs': "/outputs/dfs/05c_winD",
    'prjDir_btwD_dfs': "/outputs/dfs/05d_btwComp",

    # downsampling
    'ds_study': ['PNI'], # list of study codes to apply downsampling to
    'ds_foi': ['T1map'], # features to downsample
    'ds_res': [0.8], # resolution (in mm) to downsample volumes to. NOTE. should be same length as ds_foi with each value corresponding to that in ds_foi 
    'ds_vol_dir': '/downsampled_vols', # name of directory within project dir root

    # analysis regions
    'ctx': True, # whether to include cortical analyses
    'surf_ctx': ['fsLR-32k', 'fsLR-5k'],
    'parcellate_ctx': 'glasser', # parcellation to use, or None if no parcellation.
    'parc_lbl_ctx': 'glasser_int', # what name to fetch for parcellation values
    'lbl_ctx': ['midthickness', 'pial', 'white'], # pial, midthick, white, etc
    'ft_ctx': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_ctx': [5, 10], # in mm
    
    'hipp': True, # whether to include hippocampal analyses
    'surf_hipp': ['den-0p5mm'],
    'parcellate_hipp': 'DK25',
    'parc_lbl_hipp': 'idx',
    'lbl_hipp': ['midthickness', "inner", "outer"], # outer, inner, midthickness, etc
    'ft_hipp': ['thickness', 'T1map'], # features: T1map, flair, thickness, FA, ADC
    'smth_hipp': [2, 5], # in mm
        
    # within study comparisons
    'col_grp': 'grp_detailed',  # column in df_demo with group labels
    'winComp_stats': ['z'], # what stats to run for within study comparisons ('z' for z-scoring, 'w' for w-scoring)
    'covars': [demographics['age'], demographics['sex']],

    'ipsiTo' : 'L', # what hemisphere for controls ipsi should be mapped to
}


# Clean data
1. [removal]     Ensure that there is at least one QC_surf-vol column with a value above 0
1. [amend]      Missing one hemisphere pair, make complimentary hemisphere NA (to prevent unbalanced analyses) 
1. [removal]     NA for all smoothed maps
1. [removal]    Missing one study (3T or 7T) for a given ID-SES combination

In [ ]:
# CLEAN DATA
importlib.reload(tsutil)

# LOAD
pth = demographics["df_pths_qc_pth"]
df_pths = pd.read_csv(pth, dtype=str)
print(f"[main] df_pths loaded from {pth}")

# i. Ensure that all cases have usable data (QC value), data for both hemis present, each subject has data for both studies
df_clean, df_cln_pth, df_rmv, df_rmv_pth = tsutil.clean_demoPths(df_pths, nStudies=2, 
                                                   save_pth=f"{specs['prjDir_root']}{specs['prjDir_outs']}", 
                                                   save_name = "03c_demoPths", verbose=False)
# note can cause duplicated rows
demographics['df_pths_qc_clean_pth'] = df_cln_pth

# 4. Analysis
## a. Downsample 7T - T1maps: check if T1map standard deviation differences are due to resolution and variation in segmentation

In [ ]:
# Downsample T1maps from 7T study -- see if T1map standard deviation differences are due to resolution
# 3 Steps:
# i. Downsample 7T maps
# ii. Compute feature maps
# iii. add map paths to df_pths
# TODO can parallelize these functions to speed up computation

importlib.reload(tsutil)

test = False
reimport_src = False
df_input_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/03c_demoPths_clean_30Nov2025-094450.csv"
verbose = False
override = False # if should override existing downsampled volumes / maps

df_volPths_saveName = "04b_dfPths_dsVols"
df_mapsPths_saveName = "04c_dfPths_dsMaps"

if 'df_clean' not in globals() or df_clean is None or reimport_src == True:
    df_clean = pd.read_csv(df_input_pth, dtype=str)
#print(f"df <{df_clean.shape}> {type(df_clean)} {df_clean.columns}")

if test:
    df_clean_iter = df_clean.sample(frac=test_frac, random_state=42)
else:
    df_clean_iter = df_clean.copy()

if test:
    print(f"TEST mode. Using fraction {test_frac} of data: {df_clean_iter.shape[0]} of {df_clean.shape[0]} rows.")
    df_volPths_saveName = "TEST_" + df_volPths_saveName
    df_mapsPths_saveName = "TEST_" + df_mapsPths_saveName

# i. Downsample
df_clean_iter = tsutil.downsample_df(df = df_clean_iter, studies = studies, specs = specs,
                             demographics = demographics,
                             df_save_name = df_volPths_saveName,
                             override = False, verbose = False)

print(f"\n" + ("="*100) + f"\n")

# ii. Map
tsutil.get_dsMaps(df = df_clean_iter, 
                  specs = specs,
                  studies = studies,
                  demographics = demographics,
                  verbose = verbose,
                  override = False)

print(f"\n" + ("="*100) + f"\n")

df_clean_ds, df_svPth = tsutil.get_dsMaps_pths_iter(df_pths = df_clean_iter, specs = specs, 
                                                    demographics = demographics, studies=studies,
                                                    save_name = df_mapsPths_saveName)


In [ ]:
# Can perform sanity check here to ensure that map values are correct: ie hemispheres are different, values are same compared to zBrains maps
# copy path to see ./debug/mapValues.ipynb

## b. Read in maps

In [ ]:
# LOAD MAPS INTO DICTIONARY LISTS
importlib.reload(tsutil)

reimport_src = False
save_name = "04d_dl_maps"
test = False

if 'df_clean_ds' not in globals() or df_clean_ds is None or reimport_src == True:
    df_svPth = f"{specs['prjDir_root'] + specs['prjDir_outs']}/04c_dfPths_dsMaps_26Nov2025-132106.csv"
    df_clean_ds = pd.read_csv(df_svPth, dtype=str)

# find all map cols
cols_L, cols_R = tsutil.get_mapCols(df_clean_ds.columns, verbose=True)

# extract maps as appropriate
if specs['ctx']:
    ctx_dl = tsutil.extractMap(df_mapPaths = df_clean_ds, cols_L = cols_L, cols_R = cols_R,
                               specs = specs, studies = studies, demographics = demographics, qc_thresh = 2,
                               save_df_pth = specs['prjDir_root'] + specs['prjDir_maps_dfs'], log_save_pth = specs['prjDir_root'] + specs['prjDir_outs'],
                               region = "cortex", verbose=True, test = test)
else:
    ctx_dl = []
print("-"*100)
if specs['hipp']:
    hipp_dl = tsutil.extractMap(df_mapPaths = df_clean_ds, cols_L = cols_L, cols_R = cols_R, 
                                specs = specs, studies = studies, demographics = demographics, qc_thresh = 2,
                                save_df_pth = specs['prjDir_root'] + specs['prjDir_maps_dfs'], log_save_pth = specs['prjDir_root'] + specs['prjDir_outs'],
                                region = "hippocampus", verbose=True, test = test)
else:
    hipp_dl = []

# Create single dl 
dl = ctx_dl + hipp_dl

len_unsmth = len([d for d in ctx_dl + hipp_dl if d['smth'] == 'NA'])
len_smth = len([d for d in ctx_dl + hipp_dl if d['smth'] != 'NA'])
print(f"\n[main] {len(dl)} dictionary items for this study-feature-label-surface pairs\n\t{len_unsmth} with smoothing == NA | {len_unsmth} with smoothing")

if save:
    out_pth = tsutil.savePickle(obj = dl, root = save_pth, name = save_name, test = test)

if toPrint:
    print("="*100)
    tsutil.print_dict(dl)

# NOTE. columns with no rows are not kept in dictionary list 

In [ ]:
idx = 37
idx_other = tsutil.get_pair(dl, idx = idx, mtch = ['region', 'surf', 'label', 'feature', 'smth', 'downsampledRed'], difdsRes=True, difStudy = True)
tsutil.printItemMetadata(dl[idx], idx=idx)
tsutil.printItemMetadata(dl[idx_other], idx = idx_other)
tsutil.print_dict(dl, idx = [idx, idx_other], df_print=False)

## c. Parcellate

In [ ]:
# Parcellate maps
importlib.reload(tsutil)
reimport_src = False
save_name = "04e_dl_maps_parcel"
test = False
verbose = False

if 'dl' not in globals() or dl is None or reimport_src:
    src_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/04d_dl_maps_30Nov2025-101936.pkl"
    dl = tsutil.loadPickle(src_pth)

#tsutil.print_dict(dl)

if specs['parcellate_ctx'] is not None or specs['parcellate_hipp'] is not None: # for each item, create a df_parc   
    region_parc = [{'region': 'cortex', 
                    'parcellate': specs.get('parcellate_ctx', False),
                    'parc_lbl': specs.get('parc_lbl_ctx', None)}, 
                   
                   {'region': 'hippocampus',
                    'parcellate': specs.get('parcellate_hipp', False),
                    'parc_lbl': specs.get('parc_lbl_hipp', None)}]
    
    # TODO. Also parcellate without summarizing accross parcels
    dl_parcel, dl_parc_pth, region_parc = tsutil.parcellate_items(dl, df_keys=['df_maps'], parcellationSpecs = region_parc, df_save_pth = specs['prjDir_root'] + specs['prjDir_parc_dfs'],
                                                    stats = ['none', 'mean'],
                                                    save_pth=f"{specs['prjDir_root']}{specs['prjDir_outs']}", save_name=save_name,
                                                    verbose=verbose, test=test)

In [ ]:
dl_parc = tsutil.loadPickle("/host/verges/tank/data/daniel/01_3T7T/z/outputs/04e_dl_maps_parcel_30Nov2025-104901.pkl")
importlib.reload(tsutil)
dl_parcel_foi = tsutil.filt_dl(dl_parc, key = 'feature', voi = ['thickness','T1map'])
tsutil.print_dict(dl_parcel_foi, df_print=False)

In [ ]:
# STATISTICS about vertices within parcels
# To help select mean or median summarization
"""
# TODO. Appears broken
import importlib
import utils_parc as up
importlib.reload(up)

reimport_src = False
if 'dl_parcel' not in globals() or dl_parcel is None or reimport_src:
    pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/04c_dl_maps_parcel_07Oct2025-112622.pkl"
    dl_parcel = tsutil.loadPickle(pth)
    print(f"[main] Dict list with parcellated map values loaded from {pth}")

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs/04e_parcel_distr"
koi = "df_maps_parc-dk25"
up.parcel_stats(dl = dl_parcel, key = koi, sv_root = save_pth, test = False)
"""


In [ ]:
# SHOW MAP MATRICES
importlib.reload(tsutil)
importlib.reload(uplots)

fig_dir = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/04f_maps_allPt/raw"
uplots.plotMatrices(dl = dl_parcel_foi, df_keys = ['df_maps_parc-glsr_mean', 'df_maps_parc-dk25_mean'], 
                    cor = True, save_pth=fig_dir, test=False) # visualize smoothed maps


In [ ]:

uplots.plotLine(dl_parcel, df_keys = ['df_maps_parc-glsr_mean', 'df_maps_parc-dk25_mean'],
            name_append="line", 
            parc=['glasser', 'DK25'], stat = ['mean', 'mean'],
            hlines = [[60,120,240,300], None],
            save_pth="/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/04f_maps_allPt/raw",
            marks = False, alpha = 0.6,
            test=False)

#tsutil.pngs2pdf(fig_dir, output="/host/verges/tank/data/daniel/3T7T/z/outputs/figs/maps_allPt") # group pngs of same comparisons with different smoothing to single pdf


# Within study, vertex/parcel-wise statistics (z-, w- scores)
- compares _all_ participants to controls 


In [ ]:
# compute z, w scores within studies (all participants vs control distribution)
importlib.reload(tsutil)
importlib.reload(uplots)

# import smoothed maps
reimport_src = False
test = False
save_name = "05a_winStudy"

if 'dl_parcel' not in globals() or dl_parcel is None or reimport_src:
    pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/04e_dl_maps_parcel_30Nov2025-104901.pkl"
    dl = tsutil.loadPickle(pth, verbose = False)
    print(f"[main] Dict list with parcellated map values loaded from {pth}")

#tsutil.print_dict(dl_parcel, df_print=False)

# calculate statistics
dl_winComp, dl_winComp_pth  = tsutil.winComp(dl = dl_parcel, demographics = demographics, keys_maps = ['df_maps', 'df_maps_parc-glsr', 'df_maps_parc-glsr_mean', 'df_maps_parc-dk25', 'df_maps_parc-dk25_mean'], 
                            col_grp = specs['col_grp'], ctrl_grp = ctrl_grp, 
                            out_df_save_pth = specs['prjDir_root'] + specs['prjDir_winComp_dfs'],
                            stat=specs['winComp_stats'], covars = specs['covars'], key_demo = 'df_demo',
                            save = True, save_pth = specs['prjDir_root'] + specs['prjDir_outs'], save_name = save_name,
                            verbose = True, dlPrint = False, test=test)


In [ ]:
# plot z, w score maps
importlib.reload(tsutil)
importlib.reload(uplots)

reimport_src = False
test = False
fig_dir = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/05a_winComp/raw"

if 'dl_winComp' not in globals() or dl_winComp is None or reimport_src:
    pth = dl_winComp_pth
    # pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/05a_winStudy_30Nov2025-110838.pkl"
    dl_winComp = tsutil.loadPickle(pth, verbose = False)
    print(f"[main] winComp dict list loaded from {pth}")

# TODO. Add smart plotting based on parameters listed in specs dictionary
dfs_toPlot = ['df_maps_parc-glsr_mean_z', 'df_maps_parc-dk25_mean_z']

uplots.plotMatrices(dl = dl_winComp, df_keys = dfs_toPlot, name_append=True, save_pth=fig_dir, test=test) # visualize winCompStat maps
uplots.plotLine(dl_winComp, df_keys = dfs_toPlot, 
            parc= ['glasser', 'DK25'], stat = ['z', 'z'],
            hlines = [[60,120,240,300], None],
            save_pth=fig_dir,
            marks = False, alpha = 0.6,
            test=test)

# TODO. Allow integration of pdf for large images
#tsutil.pngs2pdf(fig_dir, output="/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05a_winComp", verbose = True) # group pngs of same comparisons with different smoothing to single pdf


# Select group of interest and ipsi/contra flip

In [ ]:
# Create new dictionary list based on previous dl.
# New dl will have the same number of dictionary items (one for each study, ft, label, surf, smth, region combination).
#   Keys of each dictionary items may change. One df for each combination of [group[len(goi)] x lateralization[_R, _L, _ic] + 1 (ctrl)] x stat[<_z>, <_w>]] 
#   If df_{stat} is none, nothing regarding this statistic will be added to dict item.

importlib.reload(tsutil)

# import
reimport_src = False
src_pth = dl_winComp_pth
#src_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/05a_winStudy_20Oct2025-084311.pkl"

if 'dl_winComp' not in globals() or dl_winComp is None or reimport_src:
    dl_winComp = tsutil.loadPickle(src_pth, verbose = True)

importlib.reload(tsutil)
goi  = ["TLE"] # Should be high level group eg., TLE. Will find lateralization based on last character of group name in df [L or R]
g_exclude = [["nTLE", "TLE_BL", "TLE_U"]] # group names to exclude if contain same strings as in goi list. should be a list of lists with same length as goi
# goi = ["TLE"] # ORIG group(s) of interest. Store main diagnosis abrev in list to allow for multiple groups
koi = ['df_maps_z', 'df_maps_parc-glsr_mean_z', 'df_maps_parc-dk25_mean_z'] # keys of dl_winComp to use
test = False
save_name = "05b_stats_winStudy_grp"
verbose = True

dl_grp_ic = tsutil.grp_flip(dl = dl_winComp, demographics = demographics, 
                            goi = goi, g_exclude = g_exclude, df_keys = koi,
                    col_grp = specs['col_grp'], save_pth_df = specs['prjDir_root'] + specs['prjDir_grpFlip_dfs'],
                    save_pth = specs['prjDir_root'] + specs['prjDir_outs'], save_name = save_name, test=test, verbose=verbose)

In [ ]:
tsutil.print_dict(dl_grp_ic, df_print=False)


In [ ]:
# visualize
importlib.reload(tsutil)
importlib.reload(uplots)

reimport_src = False
src_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/05b_stats_winStudy_grp_30Nov2025-114034.pkl"
printDl = True

test = False
dfs_toPlot = ['df_maps_parc-glsr_mean_z_TLE_ic', 'df_maps_parc-dk25_mean_z_TLE_ic']
foi = ["thickness", "T1map"] # features of interest
loi = ['midthickness', 'white', 'inner', 'outer'] # surfaces of interest
fig_dir = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/05b_winStudy_grp/raw"


if 'dl_grp_ic' not in globals() or dl_grp_ic is None or reimport_src:
    dl_grp_ic = tsutil.loadPickle(src_pth, verbose = True)

dl_interest = [d for d in dl_grp_ic if d['feature'] in foi and d['label'] in loi]

if printDl:
    print("="*100)
    tsutil.print_dict(dl_interest, df_print=False)

uplots.plotMatrices(dl = dl_interest, df_keys = dfs_toPlot, 
                    name_append=True, save_pth=fig_dir, test=test) # visualize z score maps

#tsutil.pngs2pdf(fig_dir, output="/host/verges/tank/data/daniel/3T7T/z/outputs/figs/05b_winStat_ic", verbose = True) # group pngs of same comparisons with different smoothing to single pdf
uplots.plotLine(dl_interest, df_keys = dfs_toPlot,
                name_append="line",
                parc= ['glasser', 'DK25'], stat = ['z', 'z'],
                hlines = [[60,120,240,300], None],
                save_pth=fig_dir, spacing = None,
                marks=False, alpha = 0.6,
                test=test)

# Within study Cohen's D

In [ ]:
importlib.reload(tsutil)

reimport = False
test = False
toPrint = False
save = True
save_name = "05c_stats_winD"

koi = ['df_maps_z',  'df_maps_parc-glsr_mean_z', 'df_maps_parc-dk25_mean_z']
goi = ['TLE_ic']

# import
pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/05b_stats_winStudy_grp_30Nov2025-114034.pkl"
if 'dl_grp_ic' not in globals() or dl_grp_ic is None or reimport == True:
    dl_grp_ic = tsutil.loadPickle(pth, dlPrint=toPrint)

winD = tsutil.winD(dl = dl_grp_ic, df_keys = koi, save_pth_df = specs['prjDir_root'] + specs['prjDir_winD_dfs'],
                   ipsiTo = specs.get('ipsiTo', 'L'), 
                   save = save, save_pth = specs['prjDir_root'] + specs['prjDir_outs'], save_name = save_name,
                   verbose = verbose, test = test)


In [ ]:
df = tsutil.loadPickle(winD[5]['df_maps_z_d_ic'])
df.index.tolist()

In [ ]:
# visualize matrices
importlib.reload(tsutil)
importlib.reload(uplots)
save_pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/05c_winD/raw"
uplots.plotMatrices(dl = winD, df_keys = 'df_d', save_pth=save_pth) # Visualize unsmoothed maps
uplots.plotMatrices(dl = winD, df_keys = 'df_d_ic', save_pth=save_pth) # Visualize unsmoothed maps
tsutil.pngs2pdf(fig_dir = save_pth, output = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/figs/05c_winD", verbose = True)

# Between study: D-score differences
- Identify pairs of dictionary items
- Extract d scoring statitics and compute:
- raw d dif
- d dif / ctrl d

In [ ]:
importlib.reload(tsutil)

reimport = False
test = False
toPrint = False
verbose = True
save_name = "05d_btwD"

# import 
pth = "/host/verges/tank/data/daniel/01_3T7T/z/outputs/05c_stats_winD_30Nov2025-114813.pkl"
if 'winD' not in globals() or winD is None or reimport == True:
    winD = tsutil.loadPickle(pth, dlPrint=toPrint)

comps = tsutil.btwD(dl = winD, koi = ["df_maps_z_d", "df_maps_z_d_ic", 
                                           "df_maps_parc-glsr_mean_z_d", "df_maps_parc-glsr_mean_z_d_ic",
                                           "df_maps_parc-dk25_mean_z_d", "df_maps_parc-dk25_mean_z_d_ic"], 
                    save_pth_df = specs['prjDir_root'] + specs['prjDir_btwD_dfs'],
                    save = save, save_pth = specs['prjDir_root'] + specs['prjDir_outs'], save_name = save_name,
                    verbose = verbose, test = test)


In [ ]:
tsutil.print_dict(comps, df_print=False)